In [2]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt 
from mpl_toolkits.axes_grid1 import make_axes_locatable

# 2019 Census block group data, 2019 NHGIS block group shapefiles, Orlando neighborhood shapefiles
block_groups = gpd.read_file('E:/.Personal Data/Misc/Docs/School/Fall 2022 (The end)/assets/FL_blck_grp_2019.shp')
neighborhoods = gpd.read_file('E:/.Personal Data/Misc/Docs/School/Fall 2022 (The end)/assets/OrlandoNeighborhoods.shp')

access_data = pd.read_csv('E:/.Personal Data/Misc/Docs/School/Fall 2022 (The end)/assets/data.csv')

# Merge census data and NHGIS shapefiles (see data-guide.txt to understand column data)
cols = ['GISJOIN', 'ALUCE001', 'AL1YE013','AL1YE001','AL1YE012', 'ALWYE002', 'ALWYE001', 'AL1ZM006']
shapes = block_groups.merge(access_data.loc[:, cols], 
                                           how='inner', 
                                           on='GISJOIN',
                                           validate='one_to_one')

# Data calculations
shapes['Internet access rate %'] = round((1 - (shapes['AL1YE013'] / (shapes['AL1YE001'] + shapes['AL1YE012'])))  * 100 , 2)
shapes['Poverty rate %'] = round((shapes['ALWYE002'] / shapes['ALWYE001']) * 100, 2)

# Select correct geography
orlando = shapes.overlay(neighborhoods.to_crs("ESRI:102003"), how="intersection")
orlando = orlando.to_crs("EPSG:4269")


In [8]:
# Label data columns
orlando = orlando.rename(columns={ 'NeighName': 'Neighborhood',
                            'ALUCE001': 'Total Population',
                            'TRACTCE': 'Census Tract', 
                            'BLKGRPCE': 'Census Block Group',
                            'AL1ZM006': 'Households Without a Computer Present'})

# Select data to be displayed
map = orlando.iloc[:,[24, 15, 21, 22, 23, 3, 4, 28]]

# Create interactive map
map_file = map.explore(column='Internet access rate %', cmap='Purples')
map_file.max_bounds = True

In [4]:
map_file

In [5]:
map_file.save('access-map.html')